In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

# --- 1. Preparação dos Dados ---
# Criando um dataset sintético para identificação de fraude com 6 classes.
# Cada classe representará um tipo de transação (legítima ou fraudulenta).
# As características (features) serão numéricas e podem representar:
# - Quantia da Transação (ex: valor em R$)
# - Frequência de Transações por ID (ex: quantas transações em um curto período)
# - Localização Geográfica (ex: proximidade de áreas de risco, ou diferença de país)
# - Horário da Transação (ex: madrugadas)
# - Tentativas de Login Falhas (para um tipo de fraude online)
# - Número de Itens na Transação

# Definindo as 6 classes de transação:
# 0: Transação Legítima (maioria)
# 1: Fraude de Cartão Clonado (transações grandes, fora do padrão)
# 2: Fraude de Identidade (muitas transações pequenas, novo padrão)
# 3: Fraude de Reembolso (transações grandes, devoluções rápidas)
# 4: Fraude de Compras Online (pequenas, repetidas, ou de alto risco)
# 5: Tentativa de Acesso Não Autorizado (não é transação, mas um evento de fraude)

np.random.seed(42) # Para reprodutibilidade

num_samples_per_class = {
    0: 1000,  # Legítima (maioria)
    1: 50,   # Cartão Clonado
    2: 70,   # Identidade
    3: 30,   # Reembolso
    4: 80,   # Compras Online
    5: 40    # Acesso Não Autorizado
}

# Criando DataFrames para cada classe com características específicas
data_frames = []

# Classe 0: Transação Legítima
df_0 = pd.DataFrame({
    'Quantia_Transacao': np.random.normal(50, 20, num_samples_per_class[0]).clip(min=1),
    'Freq_Transacoes_ID': np.random.randint(1, 5, num_samples_per_class[0]),
    'Risco_Localizacao': np.random.uniform(0.1, 0.4, num_samples_per_class[0]),
    'Horario_Risco': np.random.uniform(0.1, 0.3, num_samples_per_class[0]), # Ex: 0-1, onde 1 é alto risco (madrugada)
    'Tentativas_Login_Falhas': np.random.randint(0, 1, num_samples_per_class[0]),
    'Num_Itens': np.random.randint(1, 5, num_samples_per_class[0]),
    'Tipo_Fraude': 0
})
data_frames.append(df_0)

# Classe 1: Fraude de Cartão Clonado
df_1 = pd.DataFrame({
    'Quantia_Transacao': np.random.normal(300, 50, num_samples_per_class[1]).clip(min=100), # Quantias maiores
    'Freq_Transacoes_ID': np.random.randint(1, 3, num_samples_per_class[1]), # Poucas transações, mas suspeitas
    'Risco_Localizacao': np.random.uniform(0.6, 0.9, num_samples_per_class[1]), # Alta de risco
    'Horario_Risco': np.random.uniform(0.6, 0.9, num_samples_per_class[1]), # Madrugada/horas incomuns
    'Tentativas_Login_Falhas': np.random.randint(0, 1, num_samples_per_class[1]),
    'Num_Itens': np.random.randint(1, 3, num_samples_per_class[1]),
    'Tipo_Fraude': 1
})
data_frames.append(df_1)

# Classe 2: Fraude de Identidade (muitas pequenas, novo padrão)
df_2 = pd.DataFrame({
    'Quantia_Transacao': np.random.normal(20, 10, num_samples_per_class[2]).clip(min=1), # Quantias menores
    'Freq_Transacoes_ID': np.random.randint(5, 15, num_samples_per_class[2]), # Alta frequência
    'Risco_Localizacao': np.random.uniform(0.5, 0.8, num_samples_per_class[2]),
    'Horario_Risco': np.random.uniform(0.4, 0.7, num_samples_per_class[2]),
    'Tentativas_Login_Falhas': np.random.randint(0, 1, num_samples_per_class[2]),
    'Num_Itens': np.random.randint(1, 3, num_samples_per_class[2]),
    'Tipo_Fraude': 2
})
data_frames.append(df_2)

# Classe 3: Fraude de Reembolso (transações grandes, devoluções rápidas)
df_3 = pd.DataFrame({
    'Quantia_Transacao': np.random.normal(500, 100, num_samples_per_class[3]).clip(min=200), # Quantias muito grandes
    'Freq_Transacoes_ID': np.random.randint(1, 2, num_samples_per_class[3]), # Poucas, mas isoladas e grandes
    'Risco_Localizacao': np.random.uniform(0.7, 0.95, num_samples_per_class[3]), # Alta de risco
    'Horario_Risco': np.random.uniform(0.5, 0.8, num_samples_per_class[3]),
    'Tentativas_Login_Falhas': np.random.randint(0, 1, num_samples_per_class[3]),
    'Num_Itens': np.random.randint(1, 2, num_samples_per_class[3]), # Geralmente poucos itens, mas caros
    'Tipo_Fraude': 3
})
data_frames.append(df_3)

# Classe 4: Fraude de Compras Online (pequenas, repetidas, ou de alto risco)
df_4 = pd.DataFrame({
    'Quantia_Transacao': np.random.normal(80, 30, num_samples_per_class[4]).clip(min=1),
    'Freq_Transacoes_ID': np.random.randint(3, 7, num_samples_per_class[4]),
    'Risco_Localizacao': np.random.uniform(0.4, 0.7, num_samples_per_class[4]), # Risco intermediário/alto
    'Horario_Risco': np.random.uniform(0.3, 0.6, num_samples_per_class[4]),
    'Tentativas_Login_Falhas': np.random.randint(0, 1, num_samples_per_class[4]),
    'Num_Itens': np.random.randint(2, 6, num_samples_per_class[4]),
    'Tipo_Fraude': 4
})
data_frames.append(df_4)

# Classe 5: Tentativa de Acesso Não Autorizado (não é transação, mas um evento de fraude)
df_5 = pd.DataFrame({
    'Quantia_Transacao': np.random.normal(0, 0.1, num_samples_per_class[5]).clip(min=0), # Quantia irrelevante (evento)
    'Freq_Transacoes_ID': np.random.randint(0, 0, num_samples_per_class[5]), # Não é transação
    'Risco_Localizacao': np.random.uniform(0.8, 1.0, num_samples_per_class[5]), # Muito alto risco (IPs suspeitos)
    'Horario_Risco': np.random.uniform(0.8, 1.0, num_samples_per_class[5]), # Madrugadas/picos de ataque
    'Tentativas_Login_Falhas': np.random.randint(3, 10, num_samples_per_class[5]), # Múltiplas falhas
    'Num_Itens': np.random.randint(0, 0, num_samples_per_class[5]), # Não aplica
    'Tipo_Fraude': 5
})
data_frames.append(df_5)


# Concatenando todos os DataFrames
df = pd.concat(data_frames, ignore_index=True)

# Embaralhar o DataFrame para garantir que as classes não estejam em ordem sequencial
df = df.sample(frac=1, random_state=42).reset_index(drop=True)

print("Exemplo do Dataset (primeiras 5 linhas):\n", df.head())
print(f"\nNúmero total de amostras: {len(df)}")
print("Distribuição das classes (Tipos de Fraude):\n", df['Tipo_Fraude'].value_counts().sort_index())

# Separando características (X) e rótulos (y)
X = df.drop('Tipo_Fraude', axis=1).values
y = df['Tipo_Fraude'].values

# Dividindo os dados em conjuntos de treino e teste.
# test_size=0.3 significa 30% para teste, 70% para treino.
# random_state=42 para reprodutibilidade da divisão.
# stratify=y é CRUCIAL para garantir que a proporção das 6 classes seja mantida
# nos conjuntos de treino e teste, especialmente porque as classes estão desbalanceadas.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

print("\n--- Dados Após Divisão ---")
print(f"Shape de X_train (treino): {X_train.shape}")
print(f"Shape de X_test (teste): {X_test.shape}")
print(f"Shape de y_train (rótulos de treino): {y_train.shape}")
print(f"Shape de y_test (rótulos de teste): {y_test.shape}")



# 2. Treinamento do Modelo (Fase de Aprendizado)
# Usaremos o Gaussian Naive Bayes, pois as características são numéricas e contínuas.

# Inicializando o classificador Gaussian Naive Bayes.
# O parâmetro 'var_smoothing' é usado para adicionar um pequeno valor à variância
# de cada característica para evitar variâncias de zero. Isso previne problemas
# de cálculo de probabilidade e torna o modelo mais robusto.
gnb = GaussianNB(var_smoothing=1e-9)

# Treinando o modelo com os dados de treinamento (X_train e y_train).
# O algoritmo calcula as médias e variâncias de cada característica
# para CADA UMA das 6 classes de fraude/transação.
# Também calcula as probabilidades a priori de cada classe.
print("\n--- Treinando o Modelo Gaussian Naive Bayes ---")
gnb.fit(X_train, y_train)
print("Modelo treinado com sucesso!")


# 3. Classificação de Novos Dados (Fase de Predição)
# Agora, usamos o modelo treinado para prever os tipos de fraude/transação no conjunto de teste.

print("\n--- Realizando Previsões ---")
# Fazendo previsões de classe para os dados de teste.
y_pred = gnb.predict(X_test)

# Obtendo as probabilidades de cada amostra pertencer a cada uma das 6 classes.
# As colunas correspondem às classes em ordem crescente (0, 1, 2, 3, 4, 5).
y_proba = gnb.predict_proba(X_test)

print(f"Primeiras 10 previsões de tipo de fraude: {y_pred[:10]}")
print(f"Probabilidades das primeiras 2 previsões (arredondadas):\n {y_proba[:2].round(3)}")

# Exemplo de previsão para uma nova transação (hipotética)
# Uma transação de alto valor, com risco de localização alto, mas sem tentativas de login.
nova_transacao = np.array([[450, 1, 0.9, 0.7, 0, 2]])
pred_nova = gnb.predict(nova_transacao)
proba_nova = gnb.predict_proba(nova_transacao)

# Mapeamento para nomes de classe mais legíveis
class_names_map = {
    0: 'Legítima',
    1: 'Cartão Clonado',
    2: 'Identidade',
    3: 'Reembolso',
    4: 'Compras Online',
    5: 'Acesso Não Autorizado'
}

print(f"\nPrevisão para nova transação: {class_names_map[pred_nova[0]]} (Classe {pred_nova[0]})")
print("Probabilidades para a nova transação:")
for i, prob in enumerate(proba_nova[0]):
    print(f"  {class_names_map[i]}: {prob:.4f}")


# 4. Avaliação do Modelo
# Avaliando o desempenho do nosso classificador de fraude multiclasse.

print("\n--- Avaliação do Modelo ---")

# Acurácia: Proporção de previsões corretas sobre o total.
accuracy = accuracy_score(y_test, y_pred)
print(f"Acurácia do Modelo: {accuracy:.4f}")

# Relatório de Classificação: Fornece precisão, recall e F1-score para cada uma das 6 classes.
# target_names mapeia os rótulos numéricos para nomes mais descritivos.
print("\nRelatório de Classificação:")
print(classification_report(y_test, y_pred, target_names=[
    'Legítima', 'Cartão Clonado', 'Identidade', 'Reembolso', 'Compras Online', 'Acesso Não Autorizado'
]))

# Matriz de Confusão: Essencial para problemas multiclasse e desbalanceados.
# Mostra o número de acertos e erros para cada par de classes.
# Linhas: Classes Reais (True Labels)
# Colunas: Classes Previstas (Predicted Labels)
conf_matrix = confusion_matrix(y_test, y_pred)
print("\nMatriz de Confusão:")
print(conf_matrix)

# Visualizando a Matriz de Confusão (muito útil para múltiplas classes)
plt.figure(figsize=(9, 8))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', cbar=False,
            xticklabels=[
                'Previsto Legítima', 'Previsto Clonado', 'Previsto Identidade',
                'Previsto Reembolso', 'Previsto Online', 'Previsto Acesso'
            ],
            yticklabels=[
                'Real Legítima', 'Real Clonado', 'Real Identidade',
                'Real Reembolso', 'Real Online', 'Real Acesso'
            ])
plt.xlabel('Rótulo Previsto')
plt.ylabel('Rótulo Verdadeiro')
plt.title('Matriz de Confusão para Identificação de Fraude (Gaussian Naive Bayes)')
plt.show()

ValueError: high <= 0